In [ ]:
!pip install JPype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
import re, os, pickle
from collections import Counter
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from string import punctuation, digits
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from string import punctuation, digits
import jpype as jp
import string

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Make sure to replace the path with the path to your specific file
file_path = '/content/drive/MyDrive/data/Data.csv'
df = pd.read_csv(file_path)

In [ ]:
def preprocess(df):

  df = df[['text','sentiment']]
  texts = df['text']

  # drop same texts
  df.drop_duplicates(subset = ['text'], inplace = True)

  df['sentiment'] = df['sentiment'].str.lower()
  df['sentiment'] = df['sentiment'].map({'sexist': 'sexist', 'insult': 'insult', 'racist': 'racist', 'profanity': 'profanity'
  , 'not-offensive': 'notoffensive', "notoffensive": "notoffensive", "not offensive" : "notoffensive","ınsult": "insult"})
    # Create dummy columns without dropping any columns
  df_dummies = pd.get_dummies(df['sentiment'], drop_first=False)

  # Concatenate the dummy columns with the original DataFrame
  df = pd.concat([df, df_dummies], axis=1)
  df = df.dropna(subset=['sentiment'])
  df.drop(columns = ['sentiment'],inplace = True)

  # Remove words that contain a '#' in them entirely
  df["text"] = df["text"].apply(lambda text: re.sub(r"#\S+", "", text))

  # Remove rows with less than 5 characters
  df = df[df['text'].str.len() > 5]

  # Remove rows with only punctuation
  df = df[~df['text'].str.contains(r'^[\W_]+$')]

  # Remove rows with only whitespace
  df = df[~df['text'].str.isspace()]

  # Remove rows with only digits
  df = df[~df['text'].str.isdigit()]

  # Apply the function to the "text" column
  df["text"] = df["text"].apply(lambda text: re.sub(r"@\S+", "", text))

  # Remove punctuation from the 'text' column
  punctuation_translator = str.maketrans('', '', string.punctuation)
  df["text"] = df["text"].apply(lambda text: text.translate(punctuation_translator))


  return df
  # df.to_csv("/content/drive/MyDrive/data/Preprocessed_data.csv", index=False)

df = preprocess(df)

In [ ]:
WPT = nltk.WordPunctTokenizer()

def lower(text):
    text=re.sub("İ","i",text)
    text = text.lower()
    return text
def resubComma(texts):
    texts= re.sub(","," ",texts)
    return texts


# We can use some of the punctuations in favor of us.
# The replace_emoticon function is doing that for us and it converts some emoticons to words.
# It is really game changing preprocessing step especially when it comes to sentiment analysis, because they directly filled with emotions.

def replace_emoticon(word):
    check_pos = re.findall(r'(?::\)|:-\)|=\)|:D|:d|<3|\(:|:\'\)|\^\^|;\)|\(-:)', word)
    check_neg = re.findall(r'(:-\(|:\(|;\(|;-\(|=\(|:/|:\\|-_-|\):|\)-:)', word)
    if check_pos:
        #word = ":)"
        word = "SMILEYPOSITIVE"
    elif check_neg:
        #word = ":("
        word = "SMILEYNEGATIVE"
    return word

def vanish_punc(text):
    regex = re.compile('[%s]' % re.escape(punctuation))
    text = regex.sub(' ', text)
    return text

def vanish_digits(text):
    text=text.strip()
    vanish_digits = str.maketrans('', '', digits)
    text=text.translate(vanish_digits)
    return text

# This step is for excited and angry writers who don't care about spelling rules.
# For example, think about this kind of comment. "Yemeeeek harikaydıııı."
# We can not expect this kind of comment can be interpreted just regularly by our neural nets.
# So we need to make it a regular comment. Like, "Yemek harikaydı". The function named dup_vanish maks exactly this.
# However, there is a trade-off. Because of we vanish all the duplicates in words, some regular words are become misspelled.
# Like, "Yemek mükemmeldi.".After dup_vanish function it becomes "Yemek mükemeldi" which is not as we wanted.
# In the next steps, we are trying to solve this little problem

def dup_vanish(s1):
     return (''.join(i for i, _ in itertools.groupby(s1)))

In [ ]:
def reverse(s):
    if len(s) == 0:
        return s
    else:
        return reverse(s[1:]) + s[0]

def checkOpennes(word):
    vowels=['a','e','i','ı','o','ö','u','ü']
    open_vowels=['e','i','ü','ö']
    close_vowels=['a','ı','o','u']
    for i in range(len(word)):
        if reverse(word)[i] in vowels:
            if reverse(word)[i] in open_vowels:
                return True
            else:
                return False
        else:
            continue
def PresentCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if reverse(word)[i] in ei:
                return 'ei'
            elif reverse(word)[i] in üö:
                return 'üö'
            elif reverse(word)[i] in aı:
                return 'aı'
            elif reverse(word)[i] in uo:
                return 'uo'
            else:
                continue

def StartCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if (word)[i] in ei:
                return 'ei'
            elif (word)[i] in üö:
                return 'üö'
            elif (word)[i] in aı:
                return 'aı'
            elif (word)[i] in uo:
                return 'uo'
            else:
                continue

In [ ]:
import xml.etree.ElementTree as ET

# This script assumes each word is equally important and increments the count for each occurrence in the XML file,
# which is a basic approach given the lack of frequency data.

import xml.etree.ElementTree as ET

# Adjust the path to your corrected XML file
xml_path = '/content/drive/MyDrive/Orhan_bilgin.xml'

# Load and parse the XML file
tree = ET.parse(xml_path)
root = tree.getroot()

# Initialize an empty dictionary
WORDS = {}

# Iterate through each SYNSET in the XML, adjusting the path for the new root
for synset in root.findall('./SYNSET'):
    # Iterate through each LITERAL in the SYNSET
    for literal in synset.findall('.//SYNONYM/LITERAL'):
        if literal.text:  # Check if text is not None and not an empty string
            word = literal.text.strip()
            # Increment the word's count in the dictionary
            WORDS[word] = WORDS.get(word, 0) + 1
        else:
            # Handle the case where literal.text is None or empty, if necessary
            # For example, you might want to log this case or simply pass
            pass


# Now, WORDS dictionary is ready for use in the rest of your functions


In [ ]:
def replaceall(s, n,a):
    occurence = s.count(n)
    alt = []
    temp = s
    for i in range(occurence):
        temp2 = temp
        for j in range(i,occurence):
            temp2 = temp2.replace(n,a,1)
            alt.append(temp2)
        temp = temp.replace(n,"!",1)
    for i in range(len(alt)):
        alt[i] = alt[i].replace("!",n)

    return alt

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    if  word in WORDS.keys():
        number = WORDS[word]
    else:
        number = 1
    if number == 0:
        number = 1
    return number / N

def correction(word):
      return max(candi(word), key=P)

def candi(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcçdefgğhıijklmnoöprsştuüvyzw'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]

    sp     = replaceall(word,'ı','i')
    sp2     = replaceall(word,'u','ü')
    sp3    = replaceall(word,'o','ö')
    sp4     = replaceall(word,'g','ğ')
    sp5     = replaceall(word,'c','ç')
    sp6     = replaceall(word,'s','ş')
    sp7     = replaceall(word,'i','ı')
    sp8     = replaceall(word,'ö','o')
    sp9     = replaceall(word,'ş','s')
    sp10     = replaceall(word,'ğ','g')
    sp11     = replaceall(word,'ç','c')
    sp12     = replaceall(word,'ü','u')
    specials=[]
    specials.extend(sp)
    specials.extend(sp2)
    specials.extend(sp3)
    specials.extend(sp4)
    specials.extend(sp5)
    specials.extend(sp6)
    specials.extend(sp7)
    specials.extend(sp8)
    specials.extend(sp9)
    specials.extend(sp10)
    specials.extend(sp11)
    specials.extend(sp12)
    return set(deletes+transposes+replaces+inserts+specials)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def print_diff(word, s):
    if not word == s:
        print(word + " --> " + s)
counter = 0

In [ ]:
ZEMBEREK_PATH = r'/content/drive/MyDrive/zemberek-full.jar'
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

The Lemmatizer
In English, there is very limited ways of making a sentence negative.(Simply, "not"). However, in Turkish there are many ways to do that like word "değil", "-me" afformative to the verb, word "yok" and many more ways. We should not remove this words and afformatives remark that negativity of sentence since we are making sentiment analysis. So I tried to this lemmatizer without removing this words and afformatives.

It is very basic and not optimized lemmatizer. Thank you for making this Zemberek library!

In [ ]:
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
TurkishSpellChecker = jp.JClass('zemberek.normalization.TurkishSpellChecker')
TurkishSentenceNormalizer = jp.JClass('zemberek.normalization.TurkishSentenceNormalizer')
Paths = jp.JClass('java.nio.file.Paths')
# Get the path to the (baseline) lookup files
lookupRoot = Paths.get(r'/content/drive/MyDrive/normalization')
# Get the path to the compressed bi-gram language model
lmPath = Paths.get(r'/content/drive/MyDrive/lm.2gram.slm')
morphology = TurkishMorphology.createWithDefaults()
# Initialize the TurkishSentenceNormalizer class
# Instantiate the morphology class with the default RootLexicon
morph = TurkishMorphology.createWithDefaults()

# Instantiate the spell checker class using the morphology instance
spell = TurkishSpellChecker(morph)

# normalizer = TurkishSentenceNormalizer(morphology, lookupRoot, lmPath)

In [ ]:
morphology = TurkishMorphology.createWithDefaults()


def lemmatizer(word,texts):
        wordList=[]
        wordList = re.sub("[^\w]", " ",  texts).split()
        if '�' in word:
            return 'question'
        pos=wordList.index(word)
        if word=="SMILEYPOSITIVE":
            return word
        if word=="SMILEYNEGATIVE":
            return word
        sakin=''
        word=correction(word)
        if len(wordList)-pos>3 and pos>2:
            for i, kelime in enumerate(wordList[pos-3:pos+4]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos>5:
            for i, kelime in enumerate(wordList[pos:pos+5]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos<=5:
            for i, kelime in enumerate(wordList[pos:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)-pos<1 and pos>3:
            for i, kelime in enumerate(wordList[pos-3:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)<3:
            for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        else:
             for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        results = morphology.analyze(word)
        lemma=[]
        form=[]
        l=[]
        m=[]
        for i, result in enumerate(results):
            form.append(str(result.formatLong()))
            lemma.append(result.getLemmas()[0])
        if len(lemma)>1:
                analysis = morphology.analyzeSentence(sakin)
                results = morphology.disambiguate(sakin, analysis).bestAnalysis()
                for i, result in enumerate(results):
                        l.append(result.getLemmas()[0])
                        m.append(result.formatLong())
                for i in range(len(m)):
                    for j in range(len(form)):
                        if m[i]==form[j]:
                            lema=lemma[j]
                            if lema=='değil':
                                return 'değil'
                            if 'Neg' in form[j] or 'WithoutHavingDoneSo' in form[j] or 'Unable' in form[j]:
                                if checkOpennes(word):
                                    return lema+'me'
                                else:
                                    return lema+'ma'
                            if 'Without' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'siz'
                                elif PresentCheck(word)=='aı':
                                    return lema+'sız'
                                elif PresentCheck(word) == 'uo':
                                    return lema+'suz'
                                else:
                                    return lema+'süz'
                            if 'With' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'li'
                                elif PresentCheck(word)=='aı':
                                    return lema+'lı'
                                elif PresentCheck(word)=='uo':
                                    return lema+'lu'
                                else:
                                    return lema+'lü'
                            else:
                                return lema
                    else:
                        continue
        elif len(lemma)==1:
            if lemma[0]=='değil':
                return lemma[0]
            if 'Neg' in form[0] or 'WithoutHavingDoneSo' in form[0] or 'Unable' in form[0]:
                 if checkOpennes(word):
                    return lemma[0]+'me'
                 else:
                    return lemma[0]+'ma'
            elif 'Without' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'siz'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'sız'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'suz'
                else:
                    return lemma[0]+'süz'
            elif 'With' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'li'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'lı'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'lu'
                else:
                    return lemma[0]+'lü'
            else:
                return lemma[0]
        else:
            return word

In [ ]:
def clean_messages(texts):
    texts=lower(texts)
    texts=resubComma(texts)
    tokens=WPT.tokenize(texts)
    text = [token for token in tokens]
    for i, word in enumerate(text):
        text[i]=dup_vanish(vanish_digits(vanish_punc(replace_emoticon((word)))))
    text = list(filter(None,text))
    metin=' '.join(text)
    for i, word in enumerate(text):
         text[i] = lemmatizer(word,metin)
    test=' '.join(map(str, text))
    return test

In [ ]:
new=[]
for index, text in enumerate(df['text'].head(10)):
  new_text = str(clean_messages(text))

  # Print the original and corrected text
  print(f"Original text #{index+1}: {text}")
  print(f"Corrected text #{index+1}: {new_text}\n")

Original text #1: hemen cep bank yapıyorum ozaman siteye çökücez seninle bugun
Corrected text #1: hemen cep bank yap zaman site çökücez sefine burun

Original text #2: geçmiş olsun fenerin anasini sik
Corrected text #2: geçmiş olgun pelerin anasini sik

Original text #3: migros adet bilet var ilgilenen varsa yazsın
Corrected text #3: miras adet bilet kar ilgilen arsa yazın

Original text #4: çok hızlı gidenlere yavaş demek için geride duruyoruz
Corrected text #4: çok hızlı git yavaş de izin perde dur

Original text #5: nolu ile fetöcü öğrencilerin tüm borcu silindi bizler yüksek kur ve faiz mağduru olduk ylsytazminat egtkonus yuksekkurmagdurlari faizaffi
Corrected text #5: dolu aile fetö öğrenci tüm boru silindir biber yüksek kur ev faiz sağduyu oluk ylsytazminat egtkonus yuksekurmagdurlari izafi

Original text #6: sevmek insanın yuregi kadarkucukse buyugunu taşıyamazsın
Corrected text #6: sev insan yuregi kadarkucukse buyugunu taşıma

Original text #7: imam şfii ra buyuruyor allahın k